In [ ]:
import requests
import re
import json
import ast

# 指定网页的 URL
url = "https://beaconcha.in/charts/validators"

# 发送 HTTP GET 请求
response = requests.get(url)

# 检查响应的状态码
if response.status_code == 200:
    # 获取网页的源代码
    html_source = response.text

    # 使用正则表达式匹配以 "name":"# of Validators","data": 开头的部分
    pattern = r'"name":"# of Validators","data":(.*?)]}'
    match = re.search(pattern, html_source)

    if match:
        # 提取匹配到的部分，包括结尾的 "}]"
        data_block = match.group(1) + ']'
        csv_file = "validator_data.csv"
        
        data = ast.literal_eval(data_block)
        # 将列表中的数据拆分为多个数对
        flattened_data = [item for sublist in data for item in sublist]

        # 将数据重新组织为数对的列表
        pairs = [(flattened_data[i], flattened_data[i+1]) for i in range(0, len(flattened_data), 2)]

        # 打开 CSV 文件以写入数据
        with open(csv_file, mode="w", newline="") as file:
            writer = csv.writer(file)

            # 写入 CSV 文件的标题行
            writer.writerow(["Timestamp", "Value"])

            # 循环遍历数据并写入 CSV 文件
            for pair in pairs:
                writer.writerow(pair)

        print(f"数据已保存到 {csv_file} 文件中。")
else:
    print("Failed to fetch data:", response.status_code)

    
#validator数据
#放弃API 直接对get返回的响应进行解析 然后通过正则匹配获取对应的数据项